In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

from transformer import Transformer, TOK_LEN, PAD_TOK, MAX_INST_LEN
from stacksyms import getFunctions, parseELF, parseDWARF, parseDirectory, collectFrameInfo

file2funcs = dict() # = parseDirectory(debugFiledir)

#debugFilepath = 'data/cross-compile-dataset/bin/static/gcc/og/parallel'
debugFiledir = 'data/cross-compile-dataset/bin/static/gcc/og'
#debugFiledir = '.'
#debugFilepath = 'data/cross-compile-dataset/bin/static/gcc/og/sum'

In [2]:
#file2funcs = parseDirectory(debugFiledir)

#functions = parseELF(debugFilepath)
#generateFeatures(functions)
#assert(not any(filter(lambda x : PAD_TOK in x, data)))

#main = list(filter(lambda x : x.name=='main', functions))[0]
#main_x = generateFeatures(main)


01/29/2021 23:21:28 INFO:parseELF:Trying to parse data/cross-compile-dataset/bin/static/gcc/og/sum as ELF
01/29/2021 23:21:28 INFO:parseDWARF:File has debug info..
01/29/2021 23:21:29 INFO:parseELF:Found 87 functions.


In [ ]:
# TODO get frame size from .eh_frame section and compare func.variables against GDB's 'info scope func.name' output
for fname in next(os.walk(debugFiledir))[2]:
    debugFilepath = debugFiledir + os.sep + fname
    with open(debugFilepath, 'rb') as fstream:
        if not fstream.peek(1)[:4].hex() == b'\x7fELF'.hex():
            continue
    functions = parseELF(debugFilepath)
    for func in functions: # TODO: frame base is not useful unfortunately.. need frame size
        print(func.name + " frame base = " + str(func.frame_base) + ", variables = " + str(func.variables))


In [33]:
for fname in next(os.walk(debugFiledir))[2]:
    debugFilepath = debugFiledir + os.sep + fname
    with open(debugFilepath, 'rb') as fstream:
        if not fstream.peek(1)[:4].hex() == b'\x7fELF'.hex():
            continue
        print('Is ELF, checking for debug info..')
        elf = ELFFile(fstream)
        print(elf.get_machine_arch())

Is ELF, checking for debug info..
x64
Is ELF, checking for debug info..
x64
Is ELF, checking for debug info..
x64
Is ELF, checking for debug info..
x64
Is ELF, checking for debug info..
x64
Is ELF, checking for debug info..
x64
Is ELF, checking for debug info..
x64
Is ELF, checking for debug info..
x64
Is ELF, checking for debug info..
x64
Is ELF, checking for debug info..
x64
Is ELF, checking for debug info..
x64
Is ELF, checking for debug info..
x64
Is ELF, checking for debug info..
x64
Is ELF, checking for debug info..
x64
Is ELF, checking for debug info..
x64
Is ELF, checking for debug info..
x64
Is ELF, checking for debug info..
x64
Is ELF, checking for debug info..
x64
Is ELF, checking for debug info..
x64
Is ELF, checking for debug info..
x64
Is ELF, checking for debug info..
x64
Is ELF, checking for debug info..
x64
Is ELF, checking for debug info..
x64
Is ELF, checking for debug info..
x64
Is ELF, checking for debug info..
x64
Is ELF, checking for debug info..
x64
Is ELF, chec

In [2]:
from elftools.elf.elffile import ELFFile
from elftools.dwarf.constants import DW_LNS_copy, DW_LNS_set_file, DW_LNE_define_file, \
                                     DW_LNS_set_basic_block, DW_LNS_set_prologue_end, DW_LNS_set_isa
from dwarf_import.model.module import Module
from dwarf_import.io.dwarf_import import DWARFDB, DWARFImporter, place_component_in_module_tree

for fname in next(os.walk(debugFiledir))[2]:
    debugFilepath = debugFiledir + os.sep + fname
    print(f'Opening {debugFilepath} to check magic..')
    with open(debugFilepath, 'rb') as fstream:
        if not fstream.peek(1)[:4].hex() == b'\x7fELF'.hex():
            continue
        print('Is ELF, checking for debug info..')
        elf = ELFFile(fstream)
        if not elf.has_dwarf_info():
            print('does not contain DWARF info')
            continue
        print('Has debug info, parsing functions..')
        dwarfInfo = elf.get_dwarf_info()
        module, importer = parseDWARF(elf)
        file2funcs[debugFilepath] = getFunctions(module)
        #handleLineprogram(dwarfInfo)
        collectFrameInfo(file2funcs[debugFilepath], elf)


02/03/2021 23:28:49 INFO:parseDWARF:File has debug info..


Opening data/cross-compile-dataset/bin/static/gcc/og/sum to check magic..
Is ELF, checking for debug info..
Has debug info, parsing functions..


02/03/2021 23:28:49 INFO:collectFrameInfo:has .eh_frames
02/03/2021 23:28:50 INFO:collectFrameInfo:Found frame info for function sysv_sum_file.
02/03/2021 23:28:50 INFO:collectFrameInfo:Found frame info for function bsd_sum_file.
02/03/2021 23:28:50 INFO:collectFrameInfo:Found frame info for function usage.
02/03/2021 23:28:50 INFO:collectFrameInfo:Found frame info for function main.
02/03/2021 23:28:50 INFO:collectFrameInfo:Found frame info for function close_stdout_set_file_name.
02/03/2021 23:28:50 INFO:collectFrameInfo:Found frame info for function close_stdout_set_ignore_EPIPE.
02/03/2021 23:28:50 INFO:collectFrameInfo:Found frame info for function close_stdout.
02/03/2021 23:28:50 INFO:collectFrameInfo:Found frame info for function fdadvise.
02/03/2021 23:28:50 INFO:collectFrameInfo:Found frame info for function fadvise.
02/03/2021 23:28:50 INFO:collectFrameInfo:Found frame info for function adjust_value.
02/03/2021 23:28:50 INFO:collectFrameInfo:Found frame info for function gro

Opening data/cross-compile-dataset/bin/static/gcc/og/nohup to check magic..
Is ELF, checking for debug info..
Has debug info, parsing functions..


02/03/2021 23:28:50 INFO:collectFrameInfo:has .eh_frames
02/03/2021 23:28:51 INFO:collectFrameInfo:Found frame info for function usage.
02/03/2021 23:28:51 INFO:collectFrameInfo:Found frame info for function main.
02/03/2021 23:28:51 INFO:collectFrameInfo:Found frame info for function set_cloexec_flag.
02/03/2021 23:28:51 INFO:collectFrameInfo:Found frame info for function dup_cloexec.
02/03/2021 23:28:51 INFO:collectFrameInfo:Found frame info for function close_stdout_set_file_name.
02/03/2021 23:28:51 INFO:collectFrameInfo:Found frame info for function close_stdout_set_ignore_EPIPE.
02/03/2021 23:28:51 INFO:collectFrameInfo:Found frame info for function close_stdout.
02/03/2021 23:28:51 INFO:collectFrameInfo:Found frame info for function fd_reopen.
02/03/2021 23:28:51 INFO:collectFrameInfo:Found frame info for function file_name_concat.
02/03/2021 23:28:51 INFO:collectFrameInfo:Found frame info for function longest_relative_suffix.
02/03/2021 23:28:51 INFO:collectFrameInfo:Found fram

...

02/03/2021 23:32:29 INFO:collectFrameInfo:Found frame info for function xalloc_die.
02/03/2021 23:32:29 INFO:collectFrameInfo:Found frame info for function bkm_scale.
02/03/2021 23:32:29 INFO:collectFrameInfo:Found frame info for function bkm_scale_by_power.
02/03/2021 23:32:29 INFO:collectFrameInfo:Found frame info for function xstrtoumax.
02/03/2021 23:32:29 INFO:collectFrameInfo:Found frame info for function c_strcasecmp.
02/03/2021 23:32:29 INFO:collectFrameInfo:Found frame info for function close_stream.
02/03/2021 23:32:29 INFO:collectFrameInfo:Found frame info for function get_charset_aliases.
02/03/2021 23:32:29 INFO:collectFrameInfo:Found frame info for function locale_charset.
02/03/2021 23:32:29 INFO:collectFrameInfo:Found frame info for function rpl_fclose.
02/03/2021 23:32:29 INFO:collectFrameInfo:Found frame info for function clear_ungetc_buffer_preserving_position.
02/03/2021 23:32:29 INFO:collectFrameInfo:Found frame info for function rpl_fflush.
02/03/2021 23:32:29 INF

Opening data/cross-compile-dataset/bin/static/gcc/og/df to check magic..
Is ELF, checking for debug info..
Has debug info, parsing functions..


02/03/2021 23:32:31 INFO:collectFrameInfo:has .eh_frames
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function me_for_dev.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function known_value.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function add_uint_with_neg_flag.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function get_field_values.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function add_to_grand_total.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function selected_fstype.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function excluded_fstype.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function add_fs_type.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function add_excluded_fs_type.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function alloc_table_row.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found fr

02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function proper_name_utf8.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function quoting_options_from_style.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function strcaseeq6.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function gettext_quote.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function quotearg_buffer_restyled.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function quotearg_n_options.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function clone_quoting_options.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function get_quoting_style.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function set_quoting_style.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function set_char_quoting.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function set_quotin

02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function open_safer.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function hash_pjw.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function get_charset_aliases.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function locale_charset.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function mb_width_aux.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function mb_copy.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function is_basic.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function knuth_morris_pratt.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function knuth_morris_pratt_multibyte.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function mbsstr.
02/03/2021 23:32:32 INFO:collectFrameInfo:Found frame info for function mem_cd_iconv.
02/03/2021 23:32:32 INFO:collectFrameInfo:Fo

In [83]:
#from elftools.dwarf.descriptions import describe_reg_name

'''should probably be named getMaxFrameSize'''
def getFrameSize(function):
    # The number we get here statically from the .eh_frame section can actually be validated using GDB:
    # ./gdb path/to/prog
    # (gdb) set confirmation off
    # (gdb) break {func.name}
    # (gdb) r
    # (gdb) rbreak .
    # (gdb) c
    # (gdb) info frame
    #  at this point "frame at 0xADDRESS_A" - "called by frame at 0xADDRESS_B" should match our number below
    funcFrameRegs = [(key, val) for d in function.frame for key, val in d.items() if type(key)==int]
    return abs(min(funcFrameRegs, key=lambda t : t[1].arg)[1].arg)

sumFuncs = file2funcs['data/cross-compile-dataset/bin/static/gcc/og/sum']
sumMain = list(filter(lambda func : func.name == 'main', sumFuncs))[0]
print([f'{func.name} => {getFrameSize(func)}' for func in sumFuncs])
#print([describe_reg_name(regno, 'x64') + " => " + str(rule.arg) for (regno, rule) in sumMainFrameRegs])



['main => 48', 'sysv_sum_file => 56', 'bsd_sum_file => 56', 'usage => 32', 'close_stdout => 24', 'close_stdout_set_ignore_EPIPE => 8', 'close_stdout_set_file_name => 8', 'fadvise => 16', 'fdadvise => 8', 'human_options => 16', 'humblock => 40', 'default_block_size => 8', 'human_readable => 56', 'group_number => 56', 'adjust_value => 8', 'set_program_name => 16', 'quote => 8', 'quote_n => 8', 'quote_mem => 8', 'quote_n_mem => 8', 'quotearg_custom_mem => 8', 'quotearg_custom => 8', 'quotearg_n_custom_mem => 40', 'quotearg_n_custom => 8', 'quotearg_n_style_colon => 32', 'quotearg_colon_mem => 8', 'quotearg_colon => 8', 'quotearg_char => 8', 'quotearg_char_mem => 32', 'quotearg_style_mem => 8', 'quotearg_style => 8', 'quotearg_n_style_mem => 40', 'quotearg_n_style => 32', 'quotearg_mem => 8', 'quotearg => 8', 'quotearg_n_mem => 8', 'quotearg_n => 8', 'quotearg_n_options => 56', 'quotearg_free => 24', 'quotearg_alloc_mem => 56', 'quotearg_alloc => 8', 'quotearg_buffer => 56', 'quotearg_buff

In [5]:
'''Doesnt appear to be super useful for stack symbolization as compilers dont seem to emit relevant information'''
def handleLineprogram(dwarfInfo):
    print('looping through compilation units..')
    for cu in dwarfInfo.iter_CUs():
        lp = dwarfInfo.line_program_for_CU(cu)
        if lp == None:
            print('DWARF info is missing a line program for this CU')
            print(cu.keys())
            continue
        cu_filename = lp['file_entry'][0].name.decode('latin-1')
        if len(lp['include_directory']) > 0:
            dir_index = lp['file_entry'][0].dir_index
            if dir_index > 0:
                idir = lp['include_directory'][dir_index - 1]
            else:
                idir = b'.'
            cu_filename = '%s/%s' % (idir.decode('latin-1'), cu_filename)
        print(f'CU: {cu_filename}')
        #print(f'File name                            Line number    Starting address')
        for entry in lp.get_entries():
            '''
            if entry.state is None:
                # Special handling for commands that don't set a new state
                if entry.command == DW_LNS_set_file:
                    file_entry = lp['file_entry'][entry.args[0] - 1]
                    if file_entry.dir_index == 0:
                        # current directory
                        print('\n./%s:[++]' % (
                            file_entry.name.decode('latin-1')))
                    else:
                        print('\n%s/%s:' % (
                            lp['include_directory'][file_entry.dir_index - 1].decode('latin-1'),
                            file_entry.name.decode('latin-1')))
                elif entry.command == DW_LNE_define_file:
                    print('%s:' % (
                        lp['include_directory'][entry.args[0].dir_index].decode('latin-1')))
                elif entry.command in [DW_LNS_set_basic_block, DW_LNS_set_prologue_end, DW_LNS_set_isa]:
                    print('=============================')
                    print(repr(entry))
                    print('=============================')
                else:
                    print('=============================')
                    print('OTHER COMMAND: ' + str(entry))
                    print('=============================')
            elif not entry.state.end_sequence:
                # readelf doesn't print the state after end_sequence
                # instructions. I think it's a bug but to be compatible
                # I don't print them too.
                if lp['version'] < 4:
                    print('%-35s  %11d  %18s' % (
                        lp['file_entry'][state.file - 1].name.decode('latin-1'),
                        state.line,
                        '0' if state.address == 0 else
                            hex(state.address)))
                else:
                    print('%-35s  %11d  %18s[%d]' % (
                        lp['file_entry'][state.file - 1].name.decode('latin-1'),
                        state.line,
                        '0' if state.address == 0 else
                            hex(state.address),
                        state.op_index))
            if entry.command == DW_LNS_copy:
                # Another readelf oddity...
                print()
            '''
            if entry.command in [DW_LNS_set_basic_block, DW_LNS_set_prologue_end, DW_LNS_set_isa]:
                print('=============================')
                print(repr(entry))
                print('=============================')
        #print(dir(lp))
        #line_entry_mapping(lp)

def line_entry_mapping(line_program):
    import collections
    filename_map = collections.defaultdict(int)
    lp_entries = line_program.get_entries()
    for lpe in lp_entries:
        if not lpe.state or lpe.state.file == 0:
            continue # TODO: instruction doesn't correspond to src
        filename = lpe_filename(line_program, lpe.state.file)
        filename_map[filename] += 1
    for filename, lpe_count in filename_map.items():
        print('%s -> %d entries' % (filename, lpe_count))

def lpe_filename(line_program, file_index):
    lp_header = line_program.header
    print(lp_header)
    file_entries = lp_header["file_entry"]
    file_entry = file_entries[file_index - 1]
    dir_index = file_entry["dir_index"]
    if dir_index == 0:
        return file_entry.name.decode()
    directory = lp_header["include_directory"][dir_index - 1]
    return os.path.join(directory, file_entry.name).decode()

    

#print(elf.get_section_by_name('.text').data())

In [ ]:
from stacksyms import parseDWARF
module, importer = parseDWARF(debugFilepath)
for cu in module.children():
    for component in cu.children():
        for func in component.functions:
            print(func)
    
dwarfDB = importer._dwarf_db
dwarfData = dwarfDB._pri
print(dir(dwarfData))
print(dwarfData._die_map)
# TODO: we could get basic block info from DWARF.. maybe
#       look at "objdump --dwarf=line execFile" and DWARF Spec Section 6.2
# UPDATE: nope, see cell above
#lineProg = dwarfData.get_line_program()
#dir(lineProg)
#types = firstUnit.types
#globals = firstUnit.variables
functions = firstUnit.functions
print([func.name for func in functions])
#scopeQueries = ['info scope ' + func for func in functions]
disasQueries = ['disas /r ' + func.name for func in functions]
gdbOut = staticGDB(debugFilepath, functions, disasQueries)
#scopeResults, disasResults = results[0:len(results)//2], results[len(results)//2:]

In [ ]:
# TODO: newCollectLocals(gdbOut, scopeQueries, functions) and newCollectDisas(gdbOut, disasQueries, functions)
#for scope, func in zip(scopeResults, functions):
#    collectLocals(scope, functions[func])
for disas, func in zip(gdbOut, functions):
    func.disas = [tuple(line.strip().split('\\t')) for line in disas[1:-1]]

In [ ]:
X, Y, Z = [], [], []
for func in functions:
    print('////////////////////////')
    print(func.name, hex(func.start), func.frame_base)
    for lvar in func.variables:
        print(lvar.name, lvar.type, "(bytesize = %d)"%lvar.type.byte_size)
        for loc in lvar.locations:
            print(hex(loc.begin) + " to " + hex(loc.end) + ": " \
            + str(loc.type)[13:]
            + str(loc.expr))
        print('')
#    X += [func] #generateFeature(func, functions)]
#    Y += [generateLabel(func, functions)]
#    Z += [generateDebugLabel(func, functions)]
#print([x+" => "+ str(y) for x,y in zip(X,Z)])

#func = 'quotearg_n_style_colon'

In [ ]:
for func in functions:
    print('////////////////////////')
    print(func.name, func.frame_base)
    for lvar in func.variables:
        print(lvar.name, lvar.type, "(bytesize = %d)"%lvar.type.byte_size)
        for loc in lvar.locations:
    #        print(loc)
            print(hex(loc.begin) + " to " + hex(loc.end) + ": " \
    #            + str(loc.type)[13:]
                + str(loc.expr))
            print('')